In [3]:
# imports
from idaes.dmf import DMF, resource, tables
from pathlib import Path
import shutil

In [ ]:
workspace_name = "pitzer_data"
# For didactic purposes, clear out any old DMF workspace
if Path(workspace_name).exists():
    shutil.rmtree(workspace_name)
# Create DMF workspace to store data
dmf = DMF(workspace_name, create=True)

In [ ]:
# Create resource representing source publication
src = resource.Resource(name="pitzer-source", type_=resource.ResourceTypes.publication)
src.v["sources"].append({"date": "1984", 
                   "doi": "10.1063/1.555709", 
                   "source": 'Pitzer, Kenneth S., J. Christopher Peiper, and R. H. Busey. '
                   '"Thermodynamic properties of aqueous sodium chloride solutions." '
                   'Journal of Physical and Chemical Reference Data 13.1 (1984): 1-102.'})
dmf.add(src)

In [ ]:
# Find tables to read
data_files = Path(".").glob("*.csv")

In [ ]:
# Create resources for each table, add to DMF, link to source publication
for df in data_files:
    name = df.stem
    tbl = tables.Table()
    tbl.read_csv(df)
    r = resource.Resource(name=name, type_=resource.ResourceTypes.tabular)
    dmf.add(r)
    # Create the link: <publication source> -- derived --> <this table>
    resource.create_relation(src, resource.Predicates.derived, r)
    print(f"created resource {name}")
# register all the relations we just created
dmf.update()

In [ ]:
# Find all the resources derived from the Pitzer publication
pub = dmf.find_one(name="pitzer-source")
for r in dmf.find_related_resources(pub, outgoing=True):
    print(f"Resource: {r.name}")

In [7]:
# Find the publication(s) for given dataset

# get the dataset
p1 = dmf.find_one(name="pitzer_1")

# get all linked publications
for r in dmf.find_related_resources(p1, outgoing=False, maxdepth=1,
                                    predicate=resource.Predicates.derived,
                                    filter_dict={"type": resource.ResourceTypes.publication}):
    print(f"Publication: {r.sources[0]}")

2021-08-07 21:22:24,138 [INFO] idaes.dmf.dmfbase: Saving configuration location to: C:\Users\DanGunter/.dmf


Publication: {'date': '1984', 'doi': '10.1063/1.555709', 'source': 'Pitzer, Kenneth S., J. Christopher Peiper, and R. H. Busey. "Thermodynamic properties of aqueous sodium chloride solutions." Journal of Physical and Chemical Reference Data 13.1 (1984): 1-102.'}
